<a href="https://colab.research.google.com/github/d7anSeR/lab5_python/blob/main/laba5_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый раздел

In [ ]:
!pip install pymorphy2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import csv
import string

import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
morph = pymorphy2.MorphAnalyzer()

In [ ]:
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
def create_columns(name_class_good: str, name_class_bad: str, df: pd.DataFrame) -> None:
    '''create columns of DataFrame'''
    df['review'] = ""
    i = 0
    for file_dataset in os.listdir("dataset"):
        path_p = os.path.join("dataset", file_dataset)
        class_name = os.listdir(path_p)
        for elem in class_name:
            name = os.path.join(path_p, elem)
            f = open(name, mode='rt', encoding='utf-8')
            data = f.read()
            df.loc[[i], 'review'] = data
            f.close()
            i += 1

In [ ]:
def lemmatize(text) -> list:
    '''lemmatization of the text'''
    new_list = []
    tt = str.maketrans(dict.fromkeys(string.punctuation))
    elem = text.translate(tt)
    if elem is not None:
        list_words = elem.split()
    for word in list_words:
        p = morph.parse(word)[0]
        new_list.append(p.normal_form)
    return new_list

In [ ]:
name_class_good = "good"
name_class_bad = "bad"
tmp = []
data = pd.read_csv('annotation', encoding='utf-16', delimiter=';',
                 names=['absolut_path', 'otnos_path', 'class_name'])
tmp.append(data)
df = pd.concat(tmp)
df = df.drop(['otnos_path'], axis=1)
print(df)

In [ ]:
remove_non_alphabets =lambda x: re.sub(r'[^a-zA-Z]',' ',x)
tokenize = lambda x: word_tokenize(x)
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]
lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

In [ ]:
create_columns(name_class_good, name_class_bad, df)
df

In [ ]:
print('Processing : [=', end='')
df['review'] = df['review'].apply(lemmatize) # [ word_tokenize(row) for row in data['email']]
print('=', end='')
df['review'] = df['review'].apply(stem)


In [ ]:

print('=', end='')
df['review'] = df['review'].apply(lambda x: ' '.join(x))
print('] : Completed', end='')

In [ ]:
df

In [ ]:
max_words = 10000
cv = CountVectorizer(max_features=max_words, stop_words='english')
sparse_matrix = cv.fit_transform(df['review']).toarray()

In [ ]:
sparse_matrix.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(df['class_name']))

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [ ]:
model = LogisticRegression()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)

In [ ]:
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()